# AWS(S3) Data Loading & Saving Using Multiprocessing Code Tutorial

<b><u>[목적]</u></b>
- S3에 축적되는 Data를 Python 환경에 불러옴
- Partitioning을 활용하여 Multiprocessing으로 여러 개의 Core를 잡아 시간을 단축시킴
    
<b><u>[Process]</u></b>
- Data Call From AWS(S3) Using Multiprocessing
- Pickle Data Saving (Faster Than .csv or .xlsx)
    
<b><u>[주의]</u></b>
- AWS(S3)에 너무 접근을 많이하게 되면 AWS에서 락을 걸기 때문에 불러오는 시간이 느려질 수 있음
- 즉, 단시간에 데이터를 너무 많이 불러오게 되면 락 걸림"

In [5]:
import os
import gc
import sys
import boto3
import pickle
import datatable as dt
import numpy as np
import pandas as pd
import multiprocessing as mp

import warnings
warnings.filterwarnings("ignore")

from io import StringIO
from tqdm.notebook import tqdm
print(">>>> # of Cores : {}".format(mp.cpu_count()))

>>>> # of Cores : 72

In [6]:
 # Analysis Date - Long Term : list(range(20200701, 20200732, 1))
days = ['20210222', '20210223']
# # of cores - Nonlinear speed up
partitioning = 5

<b><u>[Connection S3]</u></b>
- boto3 Package를 활용한 S3 연결
- bucket, aws_access_key_id, aws_secret_access_key, region는 보안상 블록
    - 개인 혹은 단체에서 사용하는 S3 Key를 아래 Cell에 입력하여 접근함

In [7]:
# AWS INFO
bucket = 'XXXX'
aws_access_key_id = 'XXXX'
aws_secret_access_key = 'XXXX'
region = 'XXXX'

client = boto3.client('s3',aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region)
s3 = boto3.resource('s3',aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region).Bucket(bucket) 

times: user 75.5 ms, sys: 30.8 ms, total: 106 ms
Wall time: 856 ms

<b><u>[File Path List Up]</u></b>
- 선택한 날짜에 속하는 데이터 Path를 모두 List에 넣어줌

In [8]:
%%time
# Data File List up
file_list = []
for day in days:
    day = str(day)
    tmp_list = [x.key for x in s3.objects.filter(Prefix='.../year={}/month={}/day={}/'.format(day[:4], day[4:6], day[6:8]))]
    file_list.extend(tmp_list)
    
print(">>>> # of files : {}".format(len(file_list)))

>>>> # of files : 288
CPU times: user 87.7 ms, sys: 8.48 ms, total: 96.2 ms
Wall time: 382 ms

<b><u>[File List Partitioning]</u></b>
- Multiprocessing을 돌리기 위해서는 File들을 Core에 할당할 수 있도록 나누어 줘야함
- 너무 많이 나누게 되면 오히려 엄청 느려지게 되기 때문에 적당한 수준으로 나눠야함

In [9]:
# File partitioning def
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out

file_list = chunkIt(file_list, partitioning)

<b><u>[Data Load & Multiprocessing]</u></b>
- S3에 있는 데이터를 Partioning 되어 있는 List내에 For Loop를 활용하여 불러옴
- 분석 Data Quality를 향상시키기 위해 Row Missing Percentage가 (100 - 65)% 인 것은 모두 날림 (65는 Hyperparameter)
- Try & Except을 활용하여 기존 포맷을 벗어나는 file을 무시하고 불러올 수 있도록 함
- tqdm을 활용하여 진행사항 모니터링 - Multiprocessing 진행 시 사용 불가능

In [10]:
%%time
def file_loader(file_list):
    # Data Load
    data = dt.Frame()
    for i, file in enumerate(file_list):
        
        try:
            # Get data From S3
            gonie = client.get_object(Bucket=bucket, Key=file)['Body'].read().decode('utf-8')

            # Read File
            tmp = dt.fread(StringIO(gonie))

            # Qulity Up of Rows
            count = dt.rowcount(tmp).to_numpy() / tmp.shape[1]

            # Cutting
            idx_rows = np.where(count >= 0.65)[0]
            tmp = tmp[idx_rows, :]

            # Rbind
            data.rbind(tmp)
            gc.collect()
        except:
            print("{} Error".format(file))
            
    # Format Convert datatable to pandas
    data = data.to_pandas()
    return data

with mp.Pool() as p:
    data = pd.concat(p.map(file_loader, file_list))
    p.close()
    p.join()
    
# Reset Index
data.reset_index(inplace=True, drop=True)
print("Data shape : {}".format(data.shape))

Data shape : (72756, 765)
CPU times: user 2.26 s, sys: 1.84 s, total: 4.1 s
Wall time: 2min 50s

<b><u>[Data Saving]</u></b>
- Data Size는 굉장히 크면 Data 저장 포맷은 pickle형식을 추천함
- CSV or Excel 포맷은 Loading & Saving 시간이 너무 오래 걸림

In [11]:
# Data Saving info
data_format = 'pickle' # 'pickle' or 'csv' or 'excel'
save_path = 'C:/Users/user/Desktop/....'

In [12]:
if data_format == 'pickle':
    with open(save_path + "TOY_DATA.pickle", 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
elif data_format == 'csv':
    data.to_csv(save_path + "TOY_DATA.csv")
elif data_forma == 'excel':
    data.to_excel(save_path + "TOY_DATA.xlsx")